In [5]:
import datetime as dt
from typing import List, Optional
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from abc import ABC, abstractmethod

In [6]:
class Analisys:
    
    def __init__(self, source: str, chapter: str|None, analisys_name: str, analisys_cost:str, analisys_comment: str,):              

        self.source    = source
        self.chapter     = chapter
        self.analisys_name   = analisys_name
        self.analisys_cost   = analisys_cost
        self.analisys_comment  = analisys_comment

    def __repr__(self) -> str:
        return f"source = {self.source}, chapter = {self.chapter}, analisys_name = {self.analisys_name}, analisys_cost = {self.analisys_cost}, analisys_comment = {self.analisys_comment}"
    
    def to_dict(self) -> dict:
        return self.__dict__


#     def to_DB(self, ...):
#         ...

#     @classmethod
#     def from_DB(cls, ...):
#         ...


This code defines a class named Analisys that represents an analysis with several attributes: source, chapter, analisys_name, analisys_cost, and analisys_comment. The class has an __init__ method that takes these attributes as arguments and initializes the corresponding instance variables. The class also has a __repr__ method that returns a string representation of the object, and a to_dict method that returns a dictionary representation of the object.

The __init__ method takes five arguments: source, chapter, analisys_name, analisys_cost, and analisys_comment. The chapter argument is annotated with the type hint str|None, which means that it can be either a string or None. The other arguments are annotated with the type hint str, which means that they should be strings.

The __repr__ method returns a string representation of the object in the format "source = {self.source}, chapter = {self.chapter}, analisys_name = {self.analisys_name}, analisys_cost = {self.analisys_cost}, analisys_comment = {self.analisys_comment}".

The to_dict method returns a dictionary representation of the object by calling the __dict__ method on the instance, which returns a dictionary containing the instance’s attributes.

In [7]:
class WebDriver(ABC):

    def __init__(self):
        self.urls: List[str] = []
            
    def _scan_page(self, url:str):
        # Прописываем нестандартный заголовок, чтобы сайт не принял нас за бота, который парсит его данные :)
        headers = requests.utils.default_headers()
        headers.update({
                    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
                    "user-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36',
            })

        response = requests.get(url=url, headers=headers)

        if not response.ok:
            raise ValueError("no content to parse")

        return BeautifulSoup(response.content.decode(), "lxml")            

    @abstractmethod
    def get_analisys(self) :  
        ...


{'source': "medart.by/",
  'chapter': 'Лаборатория, Анализ крови',
  'analisys_name': 'Общий анализ крови из вены',
  'analisys_cost': '24.17 руб.',
  'analisys_comment': 'https://medart.by/price'+'/services/analiz-krovi/obshchiy-analiz-krovi-iz-veny/'},

In [56]:
class DriverMakayonka(WebDriver):
    
    URL = "https://makaenka17med.by/prices/"
    NAME = "makaenka17med.by"

    def __init__(self):
        ...
        
    def get_analisys(self) :
        analisys_dict={}
        i=0
        soup = self._scan_page(self.URL)
        for tag in soup.find_all(name="div", class_ = "dropdown-item"):
            analisys_comment = ""
            chapter = tag.find(name="button").text

            for row in tag.find(name="tbody").find_all("tr"):
                if row.find(name="div", class_="table-list__title"):
                    analisys_comment = row.find(name="div", class_="table-list__title").text
                else: 
                    columns = row.find_all("td")

                    analisys_name = columns[0].text
                    analisys_cost = columns[1].text
                    analisys = Analisys(self.NAME, chapter, analisys_name, analisys_cost, analisys_comment)
                    analisys_dict[i] = (analisys.to_dict())
                    i += 1

        return analisys_dict

In [11]:
class DriverMedart(WebDriver):
    
    URL = "https://medart.by/price/"
    NAME = "medart.by/"

    def get_analisys(self) :
        
        analisys_dict={}
        i=0
        soup = self._scan_page(self.URL)

        ID = ["sect_6", "sect_5", "sect_7", "sect_8", "sect_9", "sect_10", "sect_11", "sect_12", "sect_13", "sect_14", "sect_15", "sect_16", "sect_17", "sect_19", "sect_146", "sect_151"]
        
        for k in ID:
            tag = soup.find('div', class_='section_filter', id=f"{k}")

            R = tag.find(class_= 'pricemain-title_left sticky').text.strip()
            V = tag.find(class_= 'title').text.strip()
            S = tag.find(class_='categoriesmain_price-table').find('table')
#             print(R,V)

            # Находим все строки таблицы, кроме первой (заголовочной)
            rows = S.find_all("tr")#[1:]
            for row in rows:
#                 Проверяем, есть ли у элемента row атрибут colspan
                if row.find("td").has_attr('colspan'):
                    V = row.find("td", class_="title").text
                    continue
    
#                 Иначе продолжаем как обычно
                
                name_elem = row.find("td", class_="name")
                if name_elem is None:
                    print("Элемент name не найден!")
                    continue
                name = name_elem.get_text(strip=True)
                print(R,V,name)

                link = row.find("td", class_="name").find('a').get('href')
                                
                price_elem = row.find("td", class_="price")
                if price_elem is None:
                    print("Элемент price не найден!")
                    continue
                price = price_elem.get_text(strip=True)
                                
                chapter = (R,V)
                analisys_name = name
                analisys_cost = price
                analisys_comment = "https://medart.by"+link
                analisys = Analisys(self.NAME, chapter, analisys_name, analisys_cost, analisys_comment)
                analisys_dict[i] = (analisys.to_dict())
                i+=1 
                print('I=',i)
                print(analisys_dict)

        return analisys_dict

m = DriverMedart()
m.get_analisys()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



AttributeError: 'NoneType' object has no attribute 'find'

In [59]:
df_med_art = pd.DataFrame.from_dict(m.get_analisys().values())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [61]:
df_med_art.tail(40)

,source,chapter,analisys_name,analisys_cost,analisys_comment
225,medart.by/,"(Стоматология, )",Ультразвуковая чистка зубов (цена за 1 зуб),5.29руб.,https://medart.by/services/stomatologiya/ultra...
226,medart.by/,"(Стоматология, )",Композитные виниры,Уточняйте по телефонуруб.,https://medart.by/services/stomatologiya/kompo...
227,medart.by/,"(Стоматология, )",Лечение корневых каналов (цена за 1),48.75руб.,https://medart.by/services/stomatologiya/leche...
228,medart.by/,"(Стоматология, )",Лечение пародонтоза,Уточняйте по телефонуруб.,https://medart.by/services/stomatologiya/leche...
229,medart.by/,"(Стоматология, )",Лечение пародонтита,Уточняйте по телефонуруб.,https://medart.by/services/stomatologiya/leche...
230,medart.by/,"(Стоматология, )",Лечение пульпита,Уточняйте по телефонуруб.,https://medart.by/services/stomatologiya/leche...
231,medart.by/,"(Стоматология, )",Лечение кариеса,Уточняйте по телефонуруб.,https://medart.by/services/stomatologiya/leche...
232,medart.by/,"(Стоматология, )",Установка пломбы,Уточняйте по телефонуруб.,https://medart.by/services/stomatologiya/ustan...
233,medart.by/,"(Стоматология, )",Временная пломба,6.20руб.,https://medart.by/services/stomatologiya/vreme...
234,medart.by/,"(Стоматология, )",Стекловолоконный штифт,20.04руб.,https://medart.by/services/stomatologiya/stekl...


{0: {'source': 'makaenka17med.by',
  'chapter': 'КОНСУЛЬТАЦИИ СПЕЦИАЛИСТОВ \n\n\n',
  'analisys_name': 'Консультация врача-специалиста второй категории, без ктегории терапевтического профиля',
  'analisys_cost': '30,33 BYN',
  'analisys_comment': 'Консультация первичная:'
<!-- class DriverMakayonka(WebDriver):
    
    URL = "https://makaenka17med.by/prices/"
    NAME = "makaenka17med.by"

    def __init__(self):
        ...
        
    def get_analisys(self) :
        analisys_dict={}
        i=0
        soup = self._scan_page(self.URL)
        for tag in soup.find_all(name="div", class_ = "dropdown-item"):
            analisys_comment = ""
            chapter = tag.find(name="button").text

            for row in tag.find(name="tbody").find_all("tr"):
                if row.find(name="div", class_="table-list__title"):
                    analisys_comment = row.find(name="div", class_="table-list__title").text
                else: 
                    columns = row.find_all("td")

                    analisys_name = columns[0].text
                    analisys_cost = columns[1].text
                    analisys = Analisys(self.NAME, chapter, analisys_name, analisys_cost, analisys_comment)
                    analisys_dict[i] = (analisys.to_dict())
                    i += 1

        return analisys_dict -->

This code defines a class called WebDriver that inherits from the ABC (Abstract Base Class) class. The WebDriver class has an __init__ method that initializes an instance variable called urls as an empty list.

The class also defines a method called _scan_page that takes a URL as an argument. This method sets up some headers for a web request, including a custom user-agent, and then uses the requests library to send a GET request to the specified URL with these headers.

If the response to this request is not successful (i.e., if the ok attribute of the response object is False), then the method raises a ValueError with the message “no content to parse”. Otherwise, it uses the BeautifulSoup library to parse the content of the response, which is first decoded, and returns the resulting BeautifulSoup object.

The class also defines an abstract method called get_analisys, which means that any concrete subclass of WebDriver must implement this method. The implementation of this method is not shown in the code you provided.

In summary, this code defines a class that can be used to send a GET request to a specific URL, parse the response using BeautifulSoup, and provides an abstract method for performing some kind of analysis on the data.

In [6]:
class DriverSynlab(WebDriver):
    
    URL = "https://www.synlab.by"
    NAME = "Synlab.by"

    def __init__(self):
        
        super().__init__()
#         self.urls : List[str] = []
        self._chapters : List[str] = []

    def scan(self):
        url = "https://www.synlab.by/ru/%D0%BF%D1%80%D0%B5%D0%B9%D1%81%D0%BA%D1%83%D1%80%D0%B0%D0%BD%D1%82"
        soup = self._scan_page(url=url)
        tag = soup.find(name="ul", class_ = "nav menu checkup-menu mod-list")
    
        for a in tag.find_all("a"):

            url_chapter = DriverSynlab.URL + a.get("href")
            chapter = a.text
            self.urls = np.append(self.urls, url_chapter)
            self._chapters = np.append(self._chapters , chapter)
            # print(f"{url_capter = }, {name_capter = }")

        return(self)


    def get_analisys(self) :
        i = 0
#         analisys_list = []
        analisys_dict = {}

        for url_chapter, chapter in zip(self.urls, self._chapters):
            soup = self._scan_page(url_chapter)
            for tag in soup.find_all("table"):

                if tag: 
                    
                    for tr in tag.find_all("tr"):

                        t_columns = tr.find_all("td")
                        analisys_name = t_columns[0].text
                        analisys_cost = t_columns[1].text
                        if len(t_columns) == 3:
                            analisys_comment = t_columns[2].text

                        analisys = Analisys(DriverSynlab.NAME, chapter, analisys_name, analisys_cost, analisys_comment)
                        analisys_dict[i] = (analisys.to_dict())
#                         analisys_list.append(analisys)
                
                        i+=1

            time.sleep(2) # ставим задержку 2 секунды после парсинга каждого раздела
        return analisys_dict

In [7]:
d=DriverSynlab()
a=d.scan()
print(a)

This code defines a class called DriverSynlab that inherits from the WebDriver class. The DriverSynlab class has two class variables, URL and NAME, which are set to the string values "https://www.synlab.by" and "Synlab.by", respectively.

The class also has an __init__ method that calls the __init__ method of its superclass (WebDriver) using the super() function. It then initializes an instance variable called _chapters as an empty list.

The class defines a method called scan that takes no arguments. This method sets a local variable called url to a specific URL, and then calls the _scan_page method of its superclass with this URL as an argument. The result of this call, which should be a BeautifulSoup object, is stored in a local variable called soup.

Next, the method uses the find method of the soup object to find the first ul element on the page that has a class attribute equal to "nav menu checkup-menu mod-list". This element is stored in a local variable called tag.

The method then enters a loop that iterates over all a elements within this tag element using the find_all method. For each of these elements, it constructs a new URL by concatenating the value of the class variable URL with the value of the href attribute of the current a element. This new URL is stored in a local variable called url_chapter.

The method also extracts the text from the current a element and stores it in a local variable called chapter. It then appends both of these values to the instance variables urls and _chapters, respectively, using the np.append function from the NumPy library.

After this loop completes, the method returns itself (i.e., the instance of the DriverSynlab class on which it was called).

The class also defines a method called get_analisys, which overrides the abstract method with the same name in its superclass (WebDriver). This method initializes two local variables, i and analisys_dict, to 0 and an empty dictionary, respectively.

It then enters a loop that iterates over pairs of values from the instance variables urls and _chapters. For each pair of values, it calls the _scan_page method of its superclass with the current value of url_chapter as an argument. The result of this call, which should be a BeautifulSoup object, is stored in a local variable called soup.

Next, it enters another loop that iterates over all <table> elements on the page using the <EUGPSCoordinates> method of the <EUGPSCoordinates> object. If any such elements are found, it enters yet another loop that iterates over all <tr> elements within each <table> element using the <EUGPSCoordinates> method.

For each <tr> element, it finds all <td> elements within it using another call to <EUGPSCoordinates>. It then extracts text from these elements and stores it in local variables called <EUGPSCoordinates>, <EUGPSCoordinates>, and (if there are three <td> elements) <EUGPSCoordinates>.

Next, it creates an instance of a class called <EUGPSCoordinates> with these values as arguments (as well as some other values), and calls its <EUGPSCoordinates> method to convert it to a dictionary. This dictionary is then added to the <EUGPSCoordinates> dictionary with <EUGPSCoordinates> as its key.

After this innermost loop completes, <EUGPSCoordinates> is incremented by 1 and a delay of 2 seconds is introduced using the <EUGPSCoordinates> function from Python’s built-in time module.

After all loops complete, this method returns the value of <EUGPSCoordinates>.

In [6]:
class DriverKVD(WebDriver):
    
    URL = "https://kvd.by/%D0%BF%D0%BB%D0%B0%D1%82%D0%BD%D1%8B%D0%B5-%D1%83%D1%81%D0%BB%D1%83%D0%B3%D0%B8/%D0%BF%D1%80%D0%B5%D0%B9%D1%81%D0%BA%D1%83%D1%80%D0%B0%D0%BD%D1%82%D1%8B-%D0%B4%D0%BB%D1%8F-%D0%B3%D1%80%D0%B0%D0%B6%D0%B4%D0%B0%D0%BD-%D1%80%D0%B5%D1%81%D0%BF%D1%83%D0%B1%D0%BB%D0%B8%D0%BA%D0%B8-%D0%B1%D0%B5%D0%BB%D0%B0%D1%80%D1%83%D1%81%D1%8C"
    NAME = "kvd.by"

    def __init__(self):
        # Описываем все имеющиеся таблицы:
        # key - количество столбцов в таблице
        # value: 0 - номер столбца, содержащего название анализа
        # value: 1 - номер столбца, содержащего цену
        self.dict_tables = {
                10 : (2, 9),
                2 : (0, 1),
                7 : (1, 6),
                8 : (1, 7)
                }

    def get_analisys(self) :
        
        analisys_dict = {}
        soup = self._scan_page(self.URL)
        i = 0
        for tag in soup.find_all(name="div", class_ = "entry spoiler"):

#             date_update = tag.find("table").find_previous("p").strong.text
#             date_update = dt.datetime.strptime(date_update, "%d.%m.%Y")

            chapter = tag.find("h3").text

        #     n_td - количество столбцов в таблице, считаю по последней строке
            n_td = len(tag.find(class_="cool-table").find_all("tr")[-1].find_all("td"))
            for row in tag.find(class_="cool-table").find_all("tr"):
                if n_td == len(row.find_all("td")):
                    t_columns = row.find_all("td")
        #             n_td = len(t_columns)
                    analisys_name = t_columns[self.dict_tables[n_td][0]].text
                    analisys_cost = t_columns[self.dict_tables[n_td][1]].text
                    analisys = Analisys(self.NAME, chapter, analisys_name, analisys_cost, '')
                    analisys_dict[i] = (analisys.to_dict())
                    i += 1
                    
        return analisys_dict


This code defines a class `DriverMakayonka` that inherits from the `WebDriver` class. The `DriverMakayonka` class has two class attributes: `URL`, which is set to the string `"https://makaenka17med.by/prices/"`, and `NAME`, which is set to the string `"makaenka17med.by"`. The class also defines an instance method `get_analisys` that takes no arguments and returns a dictionary.

The `get_analisys` method creates an empty dictionary `analisys_dict` and a counter variable `i` initialized to 0. It then calls the `_scan_page` method with the `URL` attribute as an argument and assigns the result to the variable `soup`. The method then iterates over all `div` elements with the class `"dropdown-item"` found in `soup`. For each such element, it extracts the text of a child `button` element and assigns it to the variable `chapter`. It then iterates over all `tr` elements found in a child `tbody` element. If a `tr` element contains a child `div` element with the class `"table-list__title"`, it extracts its text and assigns it to the variable `analisys_comment`. Otherwise, it extracts the text of two child `td` elements and assigns them to the variables `analisys_name` and `analisys_cost`, respectively. It then creates an instance of the `Analisys` class with arguments `self.NAME`, `chapter`, `analisys_name`, `analisys_cost`, and `analisys_comment`, calls its `to_dict` method, and adds the result to the dictionary `analisys_dict` with key `i`. Finally, it increments the counter variable `i`.

After all iterations are complete, the method returns the dictionary `analisys_dict`.

Is there anything else you would like me to explain? 😊

In [71]:
class DriverMakayonka(WebDriver):
    
    URL = "https://makaenka17med.by/prices/"
    NAME = "makaenka17med.by"

    def __init__(self):
        ...
        
    def get_analisys(self) :
        analisys_dict={}
        i=0
        soup = self._scan_page(self.URL)
        for tag in soup.find_all(name="div", class_ = "dropdown-item"):
            analisys_comment = ""
            chapter = tag.find(name="button").text

            for row in tag.find(name="tbody").find_all("tr"):
                print('row - for row in tag.find(name="tbody").find_all("tr"):----  ',row)
                if row.find(name="div", class_="table-list__title"):
                    analisys_comment = row.find(name="div", class_="table-list__title").text
                else: 
                    columns = row.find_all("td")

                    analisys_name = columns[0].text
                    analisys_cost = columns[1].text
                    analisys = Analisys(self.NAME, chapter, analisys_name, analisys_cost, analisys_comment)
                    analisys_dict[i] = (analisys.to_dict())
                    i += 1

        return analisys_dict
        

In [69]:
maka = DriverMakayonka()

In [70]:
dict_maka = maka.get_analisys()
dict_maka

row - for row in tag.find(name="tbody").find_all("tr"): <tr>
<td class="table-list__title-wrapper" colspan="3">
<div class="table-list__title">Консультация первичная:</div>
</td>
</tr>
row - for row in tag.find(name="tbody").find_all("tr"): <tr>
<td>Консультация врача-специалиста второй категории, без ктегории терапевтического профиля</td><td>30,33 BYN</td><td>53,24 BYN</td></tr>
row - for row in tag.find(name="tbody").find_all("tr"): <tr>
<td>Консультация врача-специалиста первой категории терапевтического профиля</td><td>31,13 BYN</td><td>54,59 BYN</td></tr>
row - for row in tag.find(name="tbody").find_all("tr"): <tr>
<td>Консультация врача-специалиста высшей категории терапевтического профиля</td><td>32,24 BYN</td><td>55,97 BYN</td></tr>
row - for row in tag.find(name="tbody").find_all("tr"): <tr>
<td>Консультация врача-специалиста КМН терапевтического профиля</td><td>48,24 BYN</td><td>63,08 BYN</td></tr>
row - for row in tag.find(name="tbody").find_all("tr"): <tr>
<td>Консультация 

{0: {'source': 'makaenka17med.by',
  'chapter': 'КОНСУЛЬТАЦИИ СПЕЦИАЛИСТОВ \n\n\n',
  'analisys_name': 'Консультация врача-специалиста второй категории, без ктегории терапевтического профиля',
  'analisys_cost': '30,33 BYN',
  'analisys_comment': 'Консультация первичная:'},
 1: {'source': 'makaenka17med.by',
  'chapter': 'КОНСУЛЬТАЦИИ СПЕЦИАЛИСТОВ \n\n\n',
  'analisys_name': 'Консультация врача-специалиста первой категории терапевтического профиля',
  'analisys_cost': '31,13 BYN',
  'analisys_comment': 'Консультация первичная:'},
 2: {'source': 'makaenka17med.by',
  'chapter': 'КОНСУЛЬТАЦИИ СПЕЦИАЛИСТОВ \n\n\n',
  'analisys_name': 'Консультация врача-специалиста высшей категории терапевтического профиля',
  'analisys_cost': '32,24 BYN',
  'analisys_comment': 'Консультация первичная:'},
 3: {'source': 'makaenka17med.by',
  'chapter': 'КОНСУЛЬТАЦИИ СПЕЦИАЛИСТОВ \n\n\n',
  'analisys_name': 'Консультация врача-специалиста КМН терапевтического профиля',
  'analisys_cost': '48,24 BYN',
  'an

In [10]:
df_maka = pd.DataFrame.from_dict(dict_maka.values())
df_maka

,source,chapter,analisys_name,analisys_cost,analisys_comment
0,makaenka17med.by,КОНСУЛЬТАЦИИ СПЕЦИАЛИСТОВ \n\n\n,Консультация врача-специалиста второй категори...,"30,33 BYN",Консультация первичная:
1,makaenka17med.by,КОНСУЛЬТАЦИИ СПЕЦИАЛИСТОВ \n\n\n,Консультация врача-специалиста первой категори...,"31,13 BYN",Консультация первичная:
2,makaenka17med.by,КОНСУЛЬТАЦИИ СПЕЦИАЛИСТОВ \n\n\n,Консультация врача-специалиста высшей категори...,"32,24 BYN",Консультация первичная:
3,makaenka17med.by,КОНСУЛЬТАЦИИ СПЕЦИАЛИСТОВ \n\n\n,Консультация врача-специалиста КМН терапевтиче...,"48,24 BYN",Консультация первичная:
4,makaenka17med.by,КОНСУЛЬТАЦИИ СПЕЦИАЛИСТОВ \n\n\n,Консультация врача-специалиста ДМН терапевтиче...,"48,88 BYN",Консультация первичная:
...,...,...,...,...,...
782,makaenka17med.by,МАНИКЮР И ПЕДИКЮР\n\n\n,Женский педикюр «Классический» комплекс,"45,00 BYN",
783,makaenka17med.by,МАНИКЮР И ПЕДИКЮР\n\n\n,Женский педикюр «Классический»+долговременное ...,"55,00 BYN",
784,makaenka17med.by,МАНИКЮР И ПЕДИКЮР\n\n\n,Мужской педикюр,"35,00 BYN",
785,makaenka17med.by,МАНИКЮР И ПЕДИКЮР\n\n\n,Долговременное покрытие,"17,00 BYN",


In [11]:
dkvd = DriverKVD()

In [12]:
dict_kvd = dkvd.get_analisys()
dict_kvd

{0: {'source': 'kvd.by',
  'chapter': 'Дерматовенерология',
  'analisys_name': 'АППАРАТНОЕ УДАЛЕНИЕ НОГТЕЙ',
  'analisys_cost': '\xa0',
  'analisys_comment': ''},
 1: {'source': 'kvd.by',
  'chapter': 'Дерматовенерология',
  'analisys_name': 'Аппаратное удаление пораженной ногтевой пластинки большого пальца стопы или кисти.',
  'analisys_cost': '12,24',
  'analisys_comment': ''},
 2: {'source': 'kvd.by',
  'chapter': 'Дерматовенерология',
  'analisys_name': 'Аппаратное удаление одной пораженной ногтевой пластинки стопы или кисти.',
  'analisys_cost': '6,78',
  'analisys_comment': ''},
 3: {'source': 'kvd.by',
  'chapter': 'Дерматовенерология',
  'analisys_name': 'Механическое удаление пораженных частей ногтевой пластинки пальца стопы (кисти)при онихомикозе.',
  'analisys_cost': '6,38',
  'analisys_comment': ''},
 4: {'source': 'kvd.by',
  'chapter': 'Дерматовенерология',
  'analisys_name': 'Комплект на 1 человека',
  'analisys_cost': '1,48',
  'analisys_comment': ''},
 5: {'source': 'k

In [13]:
df_kvd = pd.DataFrame.from_dict(dict_kvd.values())
with pd.option_context('display.max_colwidth', None):
    print(df_kvd.analisys_name.head(30))

0                                                                                     АППАРАТНОЕ УДАЛЕНИЕ НОГТЕЙ
1                             Аппаратное удаление пораженной ногтевой пластинки большого пальца стопы или кисти.
2                                       Аппаратное удаление одной пораженной ногтевой пластинки стопы или кисти.
3                Механическое удаление пораженных частей ногтевой пластинки пальца стопы (кисти)при онихомикозе.
4                                                                                         Комплект на 1 человека
5                                          Одноразовый комплект для пациента (по желанию пациента дополнительно)
6                                                                                       Прием и регистрация проб
7                                           Приём, регистрация и сортировка проб в централизованных лабораториях
8                                         Взятие биологического материала с помощью транспортных

In [14]:
dSynlab = DriverSynlab()

In [15]:
dSynlab.scan()

In [16]:
lst = dSynlab.get_analisys()

In [ ]:
df_synlab = pd.DataFrame.from_dict(lst.values())
df_synlab[100:200]

,source,chapter,analisys_name,analisys_cost,analisys_comment
100,Synlab.by,БИОХИМИЧЕСКИЕ ИССЛЕДОВАНИЯ,ЛПВП-холестерин (HDL),"2,69",Жировой обмен
101,Synlab.by,БИОХИМИЧЕСКИЕ ИССЛЕДОВАНИЯ,ЛПНП-холестерин (LDL),"2,50",Жировой обмен
102,Synlab.by,БИОХИМИЧЕСКИЕ ИССЛЕДОВАНИЯ,Липопротеин А,27.67,Жировой обмен
103,Synlab.by,БИОХИМИЧЕСКИЕ ИССЛЕДОВАНИЯ,Аполипопротеин А1 (APO-A1),6.40,Жировой обмен
104,Synlab.by,БИОХИМИЧЕСКИЕ ИССЛЕДОВАНИЯ,Аполипопротеин B (APO-B),5.54,Жировой обмен
...,...,...,...,...,...
195,Synlab.by,ГОРМОНАЛЬНЫЕ ИССЛЕДОВАНИЯ,Anti-TPO (Антитела к тиреопероксидазе),15. 64,Лабораторная оценка функции щитовидной железы
196,Synlab.by,ГОРМОНАЛЬНЫЕ ИССЛЕДОВАНИЯ,Anti-TSHR (Антитела к рецепторам TSH ),34.54,Лабораторная оценка функции щитовидной железы
197,Synlab.by,ГОРМОНАЛЬНЫЕ ИССЛЕДОВАНИЯ,Кальцитонин,55.54,Лабораторная оценка функции щитовидной железы
198,Synlab.by,ГОРМОНАЛЬНЫЕ ИССЛЕДОВАНИЯ,Паратгормон,22.19,Лабораторная оценка функции щитовидной железы


In [7]:
df = pd.concat([df_kvd, df_maka, df_synlab])
df

NameError: name 'df_kvd' is not defined

In [6]:
mask = df['analisys_name'].str.contains('D') | df['analisys_name'].str.contains('АЛТ')
filtered_df = df[mask]
filtered_df

NameError: name 'df' is not defined